# Binance future tickers to onchain mappings

In this notebook, we show the corresponding onchain asset details for Binance futures. Not all Binance futures are supported on chain, but the ones that are are listed below.

In [ ]:
# https://api.binance.com/api/v3/exchangeInfo

from tradingstrategy.binance.downloader import BinanceDownloader

downloader = BinanceDownloader()
_margin_assets = downloader.fetch_assets('MARGIN')
margin_assets = list(_margin_assets)
print(len(margin_assets))

## Future mapping format

Every key in the dictionary has one or more items that adhere to the following format: 

Key: Binance ticker (see the `margin_assets` variable created above)
Value: Item or list of items where each item has 7 values: 

    (chain_id, exchange, address, tvl, swap fee, reserve_currency, has_lending, total_supplied)

- In some cases, both assets are volatile, instead of the typical case where one is volatile and the other is stable. In these cases, the `reserve_currency` just set to a placeholder value.
- The `tvl` and `total_supplied` values were recorded as at 14/15-05-2024. 

### Lending information

- Aave does not exist on binance smart chain and we don't have data for Avalanche, so we only check ethereum, polygon and arbitrum
- `has_lending is True` if both assets in the trading pair are listed on Aave. In most of the cases, at least on the tokens (typically a stable coin) is listed on Aave, but not always both tokens.
- `total_supplied` is the total amount of the asset supplied to Aave for the `base_token` when the `quote_token` is a stable coin. When both assets are volatile, which is only relevant if `has_lending = True`, only happens when ETH is one of the tokens. The non-ETH token liquidity is given. E.g. for `ETHBTC`, the `total_supplied` is the total amount of BTC supplied to Aave. So, basically, we assume the that stable coins and ETH always have decent liquidity and so don't provide figures for these. Also, we prefer Aave V3 over V2.
- USDC, USDT, DAI are on Aave except TUSD, FDUSD, BUSD

In [ ]:
from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.strategy_module import ReserveCurrency 

futures_mappings = {
    "ETHBTC": [
        (ChainId.ethereum, 'sushi', '0xceff51756c56ceffca006cd410b03ffc46dd3a58', "33.7M", '0.3%', 'BTC', True, 2_030),
        (ChainId.ethereum, 'uniswap-v2', '0xbb2b8038a1640196fbe3e38816f3e67cba72d940', '11.87M', "0.3%", 'BTC', True, 2_030),
        (ChainId.polygon, 'quickswap', '0xdc9232e2df177d7a12fdff6ecbab114e2231198d', "2.95M", "0.3%", 'BTC', True, 110.59),
        (ChainId.binance, 'pancakeswap-v2', '0xd171b26e4484402de70e3ea256be5a2630d7e88d', "2.41M", "0.25%", 'BTC', False),
        (ChainId.arbitrum, 'sparta-dex', "0xdd90a3da9688ccca100dd4da1786677bc459f069", "1.22M", "0.3%", 'BTC', True, 226.34),
        (ChainId.arbitrum, 'sushi', '0x515e252b2b5c22b4b2b6df66c2ebeea871aa4d69', '431K', '0.3%', 'BTC', True, 226.34),
    ],
    "BNBBTC": (ChainId.binance, 'pancakeswap-v2', '0x61eb789d75a95caa3ff50ed7e47b96c132fec082', '3.14M', '0.25%', False),
    "BNBETH": (ChainId.binance, 'pancakeswap-v2', '0x74e4716e431f45807dcf19f284c7aa99f18a4fbc', '3.95M', '0.25%', False),
    "BTCUSDT": [
        (ChainId.ethereum, "uniswap-v3", '0x9db9e0e53058c89e5b94e29621a205198648425b', "9.71M", "0.3%", ReserveCurrency.usdt, True, 2_030),
        (ChainId.ethereum, "uniswap-v3", '0x99ac8ca7087fa4a2a1fb6357269965a2014abc35', "20.00M", "0.3%", ReserveCurrency.usdc, True, 2_030),
        (ChainId.ethereum, "uniswap-v3", '0x9a772018fbd77fcd2d25657e5c547baff3fd7d16', "5.85M", "0.05%", ReserveCurrency.usdc, True, 2_030),
        (ChainId.arbitrum, "uniswap-v3", '0x5969efdde3cf5c0d9a88ae51e47d721096a97203', "2.2M", "0.05%", ReserveCurrency.usdt, True, 226.34),
        (ChainId.binance, "pancakeswap-v2", '0x5840b79d19c053a79923df738a251426d8879308', "1.36M", "0.25%", ReserveCurrency.usdt, False),
        (ChainId.arbitrum, "uniswap-v3", '0xac70bd92f89e6739b3a08db9b6081a923912f73d', "262.26K", "0.05%", ReserveCurrency.usdc, True, 226.34),
        (ChainId.polygon, "quickswap", "0xf6a637525402643b0654a54bead2cb9a83c8b498", "282K", "0.3%", ReserveCurrency.usdc, True, 110.38),
    ],
    "ETHUSDT": [
        (ChainId.ethereum, "uniswap-v3", '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640', '151.99M', "0.05%", ReserveCurrency.usdc, True),
        (ChainId.arbitrum, "uniswap-v3", '0x641c00a822e8b671738d32a431a4fb6074e5c79d', '12.38M', "0.05%", ReserveCurrency.usdt, True),
        (ChainId.arbitrum, "uniswap-v3", '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443', '9.26M', "0.05%", ReserveCurrency.usdc, True),
        (ChainId.binance, "biswap", '0x63b30de1a998e9e64fd58a21f68d323b9bcd8f85', '3.18M', "0.3%", ReserveCurrency.usdt, False),
    ],
    "LINKETH": (ChainId.ethereum, 'uniswap-v2', "0xa2107fa5b38d9bbd2c461d6edf11b11a50f6b974", '2.63M', '0.3%', 'ETH', True, 146.27),
    "XRTETH": (ChainId.ethereum, 'uniswap-v2', "0x3185626c14acb9531d19560decb9d3e5e80681b1", '846K', '0.3%', 'ETH', False),
    "BNBUSDT": [
        (ChainId.binance, "pancakeswap-v2", "0x16b9a82891338f9ba80e2d6970fdda79d1eb0dae", "14.00M", "0.25%", ReserveCurrency.usdt, False),
        (ChainId.binance, "pancakeswap-v2", "0xd99c7f6c65857ac913a8f880a4cb84032ab2fc5b", "727.80K", "0.25%", ReserveCurrency.usdc, False),
    ],
    "LTCUSDT": (ChainId.binance, "biswap", "0x412b607f4cbe9cae77c6f720a701cd60fa0ebd3f", "396.92K", "0.3%", ReserveCurrency.usdt, False),
    "TUSDUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0xad397a0472503b066ab4b311d66fa1f659f4cb61", "192.42K", "0.05%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x69ef438a28e6d123f809c9be785c79745ab5a0b8", "144K", "1%", ReserveCurrency.usdt, False),
    ],
    # "IOTXETH": (ChainId.ethereum, "uniswap-v3", "0x489cebe6cd5dc5dcb7047a1f0d4f358a5d2fb295", None, "0.3%"), # no tvl data
    "TRXUSDT": [
        (ChainId.ethereum, "uniswap-v2", "0x9a84a1852bc7fb608794960960adb04666a12b41", "399K", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.binance, "pancakeswap-v2", "0x1f7df58c60a56bc8322d3e42d7d37a0383d42746", "143K", "0.25%", ReserveCurrency.usdt, False),
    ],
    "USDCUSDT": (ChainId.arbitrum, "uniswap-v3", "0xbe3ad6a5669dc0b8b12febc03608860c31e2eef6", "0.01%", ReserveCurrency.usdt, True),
    "LINKUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0xfad57d2039c21811c8f2b5d5b65308aa99d31559", "588K", "0.3%", ReserveCurrency.usdc, True, 146.27),
        (ChainId.ethereum, "uniswap-v3", "0xac5a2c404ebba22a869998089ac7893ff4e1f0a7", "100K", "0.3%", ReserveCurrency.usdt, True, 146.27),
    ],
    "DOGEUSDT": (ChainId.ethereum, "uniswap-v2", "0xfcd13ea0b906f2f87229650b8d93a51b2e839ebd", "3.38M", "0.3%", ReserveCurrency.usdt, False),
    "DUSKUSDT": (ChainId.ethereum, "uniswap-v3", "0xff29d3e552155180809ea3a877408a4620058086", "789K", "0.3%", ReserveCurrency.usdt, False),
    "WANUSDT": (ChainId.ethereum, "sushi", "0x18c12a654f982f68a91e8e6a4170ca1fcb9d60a2", "126K", "0.3%", ReserveCurrency.usdt, False),
    "FUNUSDT": (ChainId.ethereum, "uniswap-v3", "0x1a349a3397a8431eed8d94a05f88f9001117fcaa", "1.65M", "0.01%", ReserveCurrency.usdt, False),
    "NKNUSDT": (ChainId.ethereum, "uniswap-v3", "0xf743ace0dd18b26ef95988486315dd7a39b07035", "116K", "0.3%", ReserveCurrency.usdt, False),
    "IOTXUSDT": (ChainId.ethereum, "sushi", "0x868d966d6d3d94b115e8b003c443c7a7b04a3359", "100k", "0.3%", ReserveCurrency.usdt, False),
    "DATAUSDT": [
        (ChainId.binance, "pancakeswap-v2", "0xe0648ec82eb04b565476b9dbc5208f180fda44a6", "140K", "0.25%", ReserveCurrency.usdt, False),
        (ChainId.polygon, "quickswap", "0x44eb80e54a22c740c919607599a3bc46750a3b71", "212K", "0.3%", ReserveCurrency.usdc, False),
    ],
    "SOLUSDT": (ChainId.binance, "babyswap", "0xae08c9d357731fd8d25681de753551be14c00405", "170K", "0.3%", ReserveCurrency.usdt, False),
    "HIVEUSDT": (ChainId.binance, "pancakeswap-v2", "0xd8eeaabc6b99df2210cd568eecd60a77e1f3fb09", "866K", "0.25%", ReserveCurrency.usdt, False),
    "SCUSDT": (ChainId.binance, "unknown", "0x1daca20a8b5904e511c4474cdfdece9fffbdbe4d", "162K", "0.3%", ReserveCurrency.usdt, False),
    "SNXUSDT": (ChainId.ethereum, "uniswap-v3", "0x020c349a0541d76c16f501abc6b2e9c98adae892", "102K", "1%", ReserveCurrency.usdc, True, 8.02),
    "BLZUSDT": (ChainId.ethereum, "uniswap-v2", "0x5fa9569b0ed6aa01e234468e6a15b77988b950df", "951K", "0.3%", ReserveCurrency.usdc, False),
    "WBTCDAI": (ChainId.ethereum, "uniswap-v3", "0x391e8501b626c623d39474afca6f9e46c2686649", "98K", "0.3%", ReserveCurrency.dai, True, 2_030),
    "ETHDAI": [
        (ChainId.ethereum, "uniswap-v3", "0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8", "25M", "0.3%", ReserveCurrency.dai, True),
        (ChainId.ethereum, "uniswap-v3", "0x60594a405d53811d3bc4766596efd80fd545a270", "8.56M", "0.05%", ReserveCurrency.dai, True),
        (ChainId.polygon, "sushi", "0x6ff62bfb8c12109e8000935a6de54dad83a4f39f", "418K", "0.3%", ReserveCurrency.dai, True),
        (ChainId.arbitrum, "uniswap-v3", "0xa961f0473da4864c5ed28e00fcc53a3aab056c1b", "264K", "0.3%", ReserveCurrency.dai, True),
    ],
    "USDTDAI": [
        (ChainId.ethereum, "uniswap-v3", "0x5777d92f208679db4b9778590fa3cab3ac9e2168", "73M", "0.01%", ReserveCurrency.usdc, True),
        (ChainId.arbitrum, "uniswap-v3", "0x7f580f8a02b759c350e6b8340e7c2d4b8162b6a9", "5.88M", "0.01%", ReserveCurrency.usdt, True),
        (ChainId.binance, "pancakeswap-v2", "0xb3c4217ab2b265bf8c69718d280e3708b5e50577", "1.06M", "0.25%", ReserveCurrency.usdt, False),
        (ChainId.polygon, "quickswap", "0xf04adbf75cdfc5ed26eea4bbbb991db002036bdd", "313K", "0.3%", ReserveCurrency.usdc, True),
        (ChainId.polygon, "uniswap-v3", "0xf369277650ad6654f25412ea8bfbd5942733babc", "282K", "0.01%", ReserveCurrency.usdc, True),
    ],
    "LUNAUSDT": (ChainId.ethereum, "uniswap-v3", "0x066375dc02ec5b1447b801ab5f5e9599dcb2a45e", "19.53M", "0.3%", ReserveCurrency.usdc, False),
    "UNIUSDT": [
        (ChainId.binance, "pancakeswpa-v2", "0xa56d15585a42adfa5ec3765c6a9434e5b06c5c4b", "742K", "0.25%", ReserveCurrency.usdt, False),
        (ChainId.ethereum, "uniswap-v3", "0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78", "356K", "0.3%", ReserveCurrency.usdc, True, 9.61),
    ],
    "SUNUSDT": [
        (ChainId.arbitrum, "unknown", "0x0cff1ab38fedee38e0d9b925d9332effe4c53e17", "2.17M", "0.3%", ReserveCurrency.usdt, False),
        (ChainId.binance, "pancakeswap-v2", "0x4c7cc204b04f5f3f1891db3f4af89895aec53c26", "708K", "0.25%", ReserveCurrency.usdt, False),
    ],
    "AVAXBTC": (ChainId.avalanche, "trader-joe", "0xd5a37dc5c9a396a03dd1136fc76a1a02b1c88ffa", "425K", "0.3%", 'BTC', False),
    "AVAXUSDT": [
        (ChainId.avalanche, "pangolin", "0x0e0100ab771e9288e0aa97e11557e6654c3a9665", "2.62M", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.avalanche, "trader-joe", "0xed8cbd9f0ce3c6986b22002f03c6475ceb7a6256", "981k", "0.3%", ReserveCurrency.usdt, False),
    ],
    "ORNUSDT": (ChainId.binance, "minereum-2", "0x684fd3c5cdfe279c82adc33029b095292d31ca35", "363K", "0.3%", ReserveCurrency.usdt, False),
    "NEARUSDT": (ChainId.ethereum, "uniswap-v3", "0xee8aaa362a75fbf8e0a38a50ddc27f38551e16ea", "656K", "0.3%", ReserveCurrency.usdc, False),
    "GRTETH": (ChainId.ethereum, "uniswap-v3", "0x2e81ec0b8b4022fac83a21b2f2b4b8f5ed744d70", "541K", "0.3%", 'ETH', False),
    "GRTUSDT": (ChainId.ethereum, "uniswap-v3", "0xb06e7ed37cfa8f0f2888355dd1913e45412798c5", "107K", "0.3%", ReserveCurrency.usdc, False),
    "1INCHUSDT": (ChainId.ethereum, "uniswap-v3", "0x9febc984504356225405e26833608b17719c82ae", "415K", "0.3%", ReserveCurrency.usdc, True, 1.10),
    "LITUSDT": (ChainId.binance, "pancakeswap-v2", "0xcda572e13ba703c8b72f0376f777d5ac0cd403a6", "3.2M", "0.25%", ReserveCurrency.usdt, False),
    "CAKEUSDT": (ChainId.binance, "pancakeswap-v2", "0xa39af17ce4a8eb807e076805da1e2b8ea7d0755b", "1.53M", "0.25%", ReserveCurrency.usdt, False),
    "BADGERBTC": (ChainId.ethereum, "uniswap-v2", "0xcd7989894bc033581532d2cd88da5db0a4b12859", "1.08M", "0.3%", 'BTC', False),
    "PONDUSDT": (ChainId.ethereum, "uniswap-v2", "0xd18748b9839b0081a867a1a871d5b562b2ec9884", "472K", "0.3%", ReserveCurrency.usdc, False),
    "ALICEUSDT": (ChainId.ethereum, "uniswap-v3", "0xde93684627d4e34d6ef96adf5bcabf10bbd8dd81", "141K", "0.3%", ReserveCurrency.usdt),
    "BAKEUSDT": (ChainId.binance, "bakeryswap", "0x6e218ea042bef40a8baf706b00d0f0a7b4fce50a", "187K", "0.3%", ReserveCurrency.busd, False),
    "MDXUSDT": (ChainId.binance, "erosswap", "0x223740a259e461abee12d84a9fff5da69ff071dd", "249K", "0.3%", ReserveCurrency.busd, False),
    "MASKUSDT": (ChainId.ethereum, "uniswap-v2", "0xa40bb1c47f6dd27142a2bd7c93bfa98db9d1f5c5", "1,27M", "0.3%", ReserveCurrency.usdc, False),
    "BONDUSDT": (ChainId.ethereum, "uniswap-v2", "0x6591c4bcd6d7a1eb4e537da8b78676c1576ba244", "234K", "0.3%", ReserveCurrency.usdc, False),
    "MLNUSDT": (ChainId.ethereum, "uniswap-v3", "0x79599e9097866e3b274c00175fd92760af5e65fa", "310K", "1%", ReserveCurrency.usdc, False),
    "FLOWUSDT": (ChainId.ethereum, "uniswap-v3", "0xd4292615a8e9edc07cba49c1a9cbb93b873a181e", "591.52K", "0.3%", ReserveCurrency.usdc, False),
    "QUICKUSDT": (ChainId.polygon, "quickswap", "0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb", "177K", "0.3%", ReserveCurrency.usdc, False),
    "REQUSDC": [
        (ChainId.ethereum, "uniswap-v2", "0xeb4b2b5e0eae7a0eadd0673ef8c3c830f8762f28", "196K", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v2", "0x2b9f8fe8ffc437a9008bb3097066f02b0a1c52ec", "185K", "0.3%", ReserveCurrency.usdt, False),
    ],
    "GHSTUSDT": (ChainId.polygon, "quickswap", "0x096c5ccb33cfc5732bcd1f3195c13dbefc4c82f4", "1.05M", "0.3%", ReserveCurrency.usdc, False), # Borrowing disabled due to an Aave community decision
    "WAXPUSDT": (ChainId.ethereum, "uniswap-v3", "0xe61cedeb1593e2d9f06af1e212842c8b5e289863", "1.04M", "0.3%", ReserveCurrency.usdt, False),
    "SOLUSDC": (ChainId.binance, "babyswap", "0xae08c9d357731fd8d25681de753551be14c00405", "168K", "0.3%", ReserveCurrency.usdc, False),
    "RADUSDT": (ChainId.ethereum, "uniswap-v2", "0x8c1c499b1796d7f3c2521ac37186b52de024e58c", "4.29M", "0.3%", ReserveCurrency.usdc, False),
    "CHESSUSDT": (ChainId.binance, "pancakeswap-v2", "0x1472976e0b97f5b2fc93f1fff14e2b5c4447b64f", "363K", "0.25%", ReserveCurrency.usdc, False),
    "ADXUSDT": (ChainId.ethereum, "uniswap-v3", "0x8de5977111c68c3fe95e63e7f7319dd5a01f77a0", "294K", "0.3%", ReserveCurrency.usdt, False),
    "BNXUSDT": (ChainId.binance, "pancakeswap-v2", "0xb72723e36a83fb5fe1793f06b436f4720f5de4f9", "3.33M", "0.25%", ReserveCurrency.busd, False),
    "ENSUSDT": (ChainId.ethereum, "uniswap-v3", "0x09aa63b7a22eefc372196aacd5b53441ed390bfb", "148K", "0.3%", ReserveCurrency.usdt, True, 3.63),
    "HIGHUSDT": [
        (ChainId.binance, "unknown", "0xcd254486fb8f5a776d8780504b87a55b80c669dc", "158K", "0.3%", ReserveCurrency.busd, False),
        (ChainId.binance, "pancakeswap-v2", "0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea", "144K", "0.25%", ReserveCurrency.busd, False),
    ],
    "GMTUSDT": [
        (ChainId.polygon, "mooar", "0xb064e9b00d713496d54617c3ef6e659072b8339f", "5.57M", "0.3%", ReserveCurrency.usdt, False),
        (ChainId.ethereum, "notdao", "0xaeaca8c32039a466fb32bde6f566130a1f49d21e", "2.03M", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.binance, "pancakeswap-v2", "0x007ec643c7cc33a70c083fc305c283dd009c8b94", "154K", "0.25%", ReserveCurrency.usdc, False),
    ],
    "BSWUSDT": (ChainId.binance, "biswap", "0x2b30c317cedfb554ec525f85e79538d59970beb0", "757K", "0.3%", ReserveCurrency.usdt, False),
    "GALUSDT": (ChainId.ethereum, "uniswap-v3", "0xf92f2e3fca01491baba0975264362cc38b1cab7b", "7.41M", "1%", ReserveCurrency.usdc, False),
    "STGUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x8592064903ef23d34e4d5aaaed40abf6d96af186", "3.13M", "1%", ReserveCurrency.usdc, False), # asset is frozen due to an Aave community decision.
        (ChainId.avalanche, "trader-joe", "0x330f77bda60d8dab14d2bb4f6248251443722009", "1.63M", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.polygon, "susi", "0xa34ec05da1e4287fa351c74469189345990a3f0c", "1.43M", "0.3%", ReserveCurrency.usdc, False),
    ],
    "HIFIUSDT": (ChainId.ethereum, "uniswap-v3", "0xc4480526e3d64f7ec7d7a6694c52d23aab658ed5", "303K", "1%", ReserveCurrency.usdc, False),
    "PROSUSDT": (ChainId.ethereum, "uniswap-v2", "0x3155acd9f75915fcc21d34035f440da7040bd3ba", "279K", "0.3%", ReserveCurrency.usdc, False),
    "AGIXUSDT": (ChainId.ethereum, "uniswap-v2", "0x4bb0925fa50da9b4c8936869433b48e78ccc5c13", "299K", "0.3%", ReserveCurrency.usdt, False),
    "GNSUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0x4d2fe06fd1c4368042b926d082484d2e3cc8f3f5", "177.64K", "1%", ReserveCurrency.dai, False),
        (ChainId.polygon, "quickswap", "0x6e53cb6942e518376e9e763554db1a45ddcd25c4", "110.57K", "0.3%", ReserveCurrency.dai, False),
    ],
    "GLMUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x60257c66cee64c535ba905dd0e7a056cbb3d8658", "330K", "1%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x60a531c09dc5b6bb82ee5c6b07ad5eed6954c929", "216K", "1%", ReserveCurrency.dai, False),
        (ChainId.ethereum, "uniswap-v3", "0x20dac1a2cf8b1bf476e144f4da439b4d8a51ea5b", "105K", "1%", ReserveCurrency.usdt, False),
    ],
    "PROMUSDT": (ChainId.ethereum, "uniswap-v3", "0x3902428a74a08a91e2cb2fd834de75e69974fe67", "175K", "0.3%", ReserveCurrency.usdt, False),
    "IDUSDT": (ChainId.ethereum, "uniswap-v3", "0x6ac6b053a2858bea8ad758db680198c16e523184", "4.11M", "0.3%", ReserveCurrency.usdc, False),
    "ARBUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0xcda53b1f66614552f834ceef361a8d12a0b8dad8", "2.87M", "0.05%", ReserveCurrency.usdc, True, 15.94),
        (ChainId.arbitrum, "uniswap-v3", "0x97bca422ec0ee4851f2110ea743c1cd0a14835a1", "246K", "0.3%", ReserveCurrency.usdt, True, 15.94),
        (ChainId.arbitrum, "uniswap-v3", "0xb791ad21ba45c76629003b4a2f04c0d544406e37", "182K", "0.05%", ReserveCurrency.usdt, True, 15.94),
    ],
    "PEPEUSDT": (ChainId.ethereum, "uniswap-v3", "0xcee31c846cbf003f4ceb5bbd234cba03c6e940c7", "153K", "1%", ReserveCurrency.usdc, False),
    "WBETHETH": (ChainId.binance, "bakeryswap", "0x5fef671df9718934fda164da289374f675745d86", "2.38M", "0.3%", ReserveCurrency.busd, False),
    "FDUSDUSDT": (ChainId.ethereum, "uniswap-v3", "0xcdfc3d54c8452b12285abb8c102df09ce83b8334", "504K", "0.01%", ReserveCurrency.usdt, False),
    "CYBERUSDT": (ChainId.ethereum, "uniswap-v3", "0x05b4285b0634de1c3e72fa3ff8d54da6fdd031d8", "351K", "1%", ReserveCurrency.usdc, False),
    "IQUSDT": (ChainId.polygon, "quickswap", "0x9f4360a2390321cb1cbff4cebeb4315d64ed3ac1", "160K", "0.3%", ReserveCurrency.usdc, False),
    "INJUSDC": (ChainId.binance, "minereum-2", "0x38c7cef2507a2434c4f831e6f850151db7e04233", "191K", "0.3%", ReserveCurrency.usdt, False),
    "MATICUSDC": [
        (ChainId.polygon, "uniswap-v3", "0xa374094527e1673a86de625aa59517c5de346d32", "2.88M", "0.05%", ReserveCurrency.usdc, True, 53.23),
        (ChainId.polygon, "mooar", "0xc84f479bf220e38ba3bd0262049bad47aaa673ee", "1.6M", "0.3%", ReserveCurrency.usdt, True, 53.23),
        (ChainId.polygon, "uniswap-v3", "0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7", "1.20M", "0.05%", ReserveCurrency.usdt, True, 53.23),
    ],
    "AIUSDT": (ChainId.binance, "unknown", "0x2f20634d0f0a1e9518f23943f4199dda049340af", "1.12M", "0.3%", ReserveCurrency.usdt, False),
    "XAIUSDT": (ChainId.ethereum, "uniswap-v3", "0x73db82e81b7a11490278687551d3b632119b0e1f", "864K", "0.3%", ReserveCurrency.usdc, False),
    "AXLUSDT": (ChainId.ethereum, "uniswap-v3", "0xae2a25cbdb19d0dc0dddd1d2f6b08a6e48c4a9a9", "9.43M", "0.05%", ReserveCurrency.usdc, False),
    "BOMEUSDT": (ChainId.binance, "pancakeswap-v2", "0xa74e6987b104947f036cffc4a56daa22300046b6", "100K", "0.25%", ReserveCurrency.usdt, False),
    "ETHFIUSDT": (ChainId.ethereum, "uniswap-v3", "0x80fa4c1fd0fbb9a4f071999af69531dee1016644", "181K", "0.3%", ReserveCurrency.usdt, False),
    "ENAUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x408a625596f47314e1fd4a6cbce84c4a8695ba3f", "362K", "0.3%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0xfc545f345e2afc52652cd6f76323c5ddc847bd6d", "107K", "0.05%", ReserveCurrency.usdc, False),
    ],
    "TAOUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0xf763bb342eb3d23c02ccb86312422fe0c1c17e94", "1.48M", "1%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x14283160198199882b5502f3316db2618baeda8c", "825K", "0.3%", ReserveCurrency.usdc, False),
    ],
}
